# Working with Events

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database, file store, and transcript usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Get all votes for a single person

Query and merge the `person` and `vote` tables.

In [5]:
# Filtering not supported for open access API yet, get all people then dataframe filter to target
people = pd.DataFrame(db.select_rows_as_list("person"))
teresa = people.loc[people["full_name"] == "Teresa Mosqueda"]

# Pull voting data
votes = pd.DataFrame(db.select_rows_as_list("vote"))

# Merge data
teresa_voting_record = teresa.merge(
    votes,
    left_on="person_id",
    right_on="person_id",
    suffixes=("_person", "_vote")
)

teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision,event_minutes_item_id,legistar_event_item_vote_id,vote_id
0,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:47:53.379896,In Favor,3460aae1-a0e8-4da4-832d-8bc3bd501495,45727,07cab2ee-29c6-441d-84dd-5a0c992f935c
1,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:04:07.362930,In Favor,49d8bd86-3a80-4218-a390-46216a0d4407,45904,0d7fb1b6-b445-470c-9c82-17f20e100ac1
2,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:03:55.459631,In Favor,a043bd70-3bd3-48ce-9fb7-75f664279953,45886,1861ce91-5f36-4332-873f-f4a9bd6994d2
3,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:04:32.261196,In Favor,99509957-9ff6-42c5-b8c1-c02fbce9e1da,45958,367bf5d3-9e64-484d-9bf4-52942e746a55
4,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:48:02.435639,In Favor,91c39ed0-4a44-4d53-81c3-44539c75ce80,45736,5d862f3b-f5ff-4bbe-b243-19e1ce9e9961


This is great, but it's missing context of what was actually being voted on. Currently we can see that she voted on instances of `event_minutes_item`, but what were those. Follow the same procedure of `select_rows_as_list` -> `merge` to join the data of the other tables to Teresa's voting record.

In [6]:
# Get event minutes item context data
event_minutes_items = pd.DataFrame(db.select_rows_as_list("event_minutes_item"))
minutes_items = pd.DataFrame(db.select_rows_as_list("minutes_item"))
events = pd.DataFrame(db.select_rows_as_list("event"))

# Merge minutes item data
event_minutes_items = event_minutes_items.merge(
    minutes_items,
    left_on="minutes_item_id",
    right_on="minutes_item_id",
    suffixes=("_event_minutes_item", "_minutes_item")
)

# Merge event data
event_minutes_items = event_minutes_items.merge(
    events,
    left_on="event_id",
    right_on="event_id",
    suffixes=("_event_minutes_item", "_event")
)

# Merge above event minutes item data with Teresa's voting record
teresa_voting_record = teresa_voting_record.merge(
    event_minutes_items,
    left_on="event_minutes_item_id",
    right_on="event_minutes_item_id",
    suffixes=("_voting_record", "_event_minutes_item")
)

# Sort by event datetime
teresa_voting_record = teresa_voting_record.sort_values(by=["event_datetime"])
teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision_voting_record,event_minutes_item_id,...,name,agenda_file_uri,body_id,created,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
5,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:41:16.563987,In Favor,8346962f-964e-49f6-8bd4-dc926126e7da,...,CB 119500,http://legistar2.granicus.com/seattle/meetings...,dd8af69a-52ae-4d0e-8cce-33494a71d89f,2019-06-04 00:40:53.106354,2019-05-28 10:30:00,3965,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/selectcommitteem...,http://video.seattle.gov:8080/media/council/ho...
0,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:47:53.379896,In Favor,3460aae1-a0e8-4da4-832d-8bc3bd501495,...,"May 28, 2019",http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
4,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:48:02.435639,In Favor,91c39ed0-4a44-4d53-81c3-44539c75ce80,...,CB 119532,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
9,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:48:07.888649,In Favor,0113c5c2-2853-4d52-ac80-b4d0e7bcd2aa,...,A RESOLUTION in support of the right to bodily...,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:47:47.028966,2019-05-28 14:00:00,3964,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
1,2019-06-04 00:03:55.213635,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,a2c03e9b-52d1-4932-9e0e-2a975d01be57,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-06-04 00:04:07.362930,In Favor,49d8bd86-3a80-4218-a390-46216a0d4407,...,CB 119513,http://legistar2.granicus.com/seattle/meetings...,e3f298eb-bb75-4273-af45-16c3f0e6f962,2019-06-04 00:03:50.288189,2019-06-03 14:00:00,3975,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
